In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("awsaf49/brats20-dataset-training-validation")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/brats20-dataset-training-validation


In [2]:
import numpy as np
import nibabel as nib
import glob
import os
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
t1_list = sorted(glob.glob('/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/*/*t1.nii'))
t2_list = sorted(glob.glob('/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/*/*t2.nii'))
t1ce_list = sorted(glob.glob('/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/*/*t1ce.nii'))
flair_list = sorted(glob.glob('/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/*/*flair.nii'))
mask_list = sorted(glob.glob('/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/*/*seg.nii'))

#Each volume generates 18 64x64x64x4 sub-volumes.
#Total 369 volumes = 6642 sub volumes
# Create the directories in the Kaggle working directory (/kaggle/working/)
os.makedirs('/kaggle/working/input_data_3channels/images', exist_ok=True) # use os.makedirs and exist_ok=True to avoid error if directory exists
os.makedirs('/kaggle/working/input_data_3channels/masks', exist_ok=True) # create masks directory
for img in range(len(t2_list)):   #Using t1_list as all lists are of same size
    print("Now preparing image and masks number: ", img)

    temp_image_t2=nib.load(t2_list[img]).get_fdata()
    temp_image_t2=scaler.fit_transform(temp_image_t2.reshape(-1, temp_image_t2.shape[-1])).reshape(temp_image_t2.shape)

    temp_image_t1ce=nib.load(t1ce_list[img]).get_fdata()
    temp_image_t1ce=scaler.fit_transform(temp_image_t1ce.reshape(-1, temp_image_t1ce.shape[-1])).reshape(temp_image_t1ce.shape)

    temp_image_flair=nib.load(flair_list[img]).get_fdata()
    temp_image_flair=scaler.fit_transform(temp_image_flair.reshape(-1, temp_image_flair.shape[-1])).reshape(temp_image_flair.shape)

    temp_mask=nib.load(mask_list[img]).get_fdata()
    temp_mask=temp_mask.astype(np.uint8)
    temp_mask[temp_mask==4] = 3  #Reassign mask values 4 to 3
    #print(np.unique(temp_mask))


    temp_combined_images = np.stack([temp_image_flair, temp_image_t1ce, temp_image_t2], axis=3)

    #Crop to a size to be divisible by 64 so we can later extract 64x64x64 patches.
    #cropping x, y, and z
    temp_combined_images=temp_combined_images[56:184, 56:184, 13:141]
    temp_mask = temp_mask[56:184, 56:184, 13:141]

    val, counts = np.unique(temp_mask, return_counts=True)

    if (1 - (counts[0]/counts.sum())) > 0.01:  #At least 1% useful volume with labels that are not 0
        print("Save Me")
        temp_mask= to_categorical(temp_mask, num_classes=4)
        # Save files to the Kaggle working directory
        np.save('/kaggle/working/input_data_3channels/images/image_'+str(img)+'.npy', temp_combined_images)
        np.save('/kaggle/working/input_data_3channels/masks/mask_'+str(img)+'.npy', temp_mask) # create masks directory

    else:
        print("I am useless")

Now preparing image and masks number:  0
Save Me
Now preparing image and masks number:  1
Save Me
Now preparing image and masks number:  2
Save Me
Now preparing image and masks number:  3
Save Me
Now preparing image and masks number:  4
I am useless
Now preparing image and masks number:  5
Save Me
Now preparing image and masks number:  6
Save Me
Now preparing image and masks number:  7
Save Me
Now preparing image and masks number:  8
Save Me
Now preparing image and masks number:  9
Save Me
Now preparing image and masks number:  10
Save Me
Now preparing image and masks number:  11
Save Me
Now preparing image and masks number:  12
Save Me
Now preparing image and masks number:  13
Save Me
Now preparing image and masks number:  14
Save Me
Now preparing image and masks number:  15
Save Me
Now preparing image and masks number:  16
Save Me
Now preparing image and masks number:  17
Save Me
Now preparing image and masks number:  18
Save Me
Now preparing image and masks number:  19
Save Me
Now p

IndexError: list index out of range

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
df=pd.read_csv("/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/survival_info.csv")
#associer les outputs npy
df['image_path'] = "/kaggle/working/input_data_3channels/images/image_" + df.index.astype(str) + '.npy'
df['mask_path'] = "/kaggle/working/input_data_3channels/masks/mask_" + df.index.astype(str) + '.npy'

In [3]:
df.columns
df.isna().sum()


,0
Brats20ID,0
Age,0
Survival_days,0
Extent_of_Resection,107
image_path,0
mask_path,0


In [4]:
df.isnull().sum()

,0
Brats20ID,0
Age,0
Survival_days,0
Extent_of_Resection,107
image_path,0
mask_path,0


In [5]:
df['Extent_of_Resection'].values


array(['GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', None, 'GTR',
       'GTR', None, 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR',
       'GTR', None, 'GTR', 'GTR', 'GTR', 'GTR', None, None, None, 'GTR',
       None, None, None, None, 'GTR', 'GTR', None, 'GTR', 'GTR', None,
       'GTR', None, None, 'GTR', None, None, None, None, None, 'GTR',
       'GTR', None, None, None, 'GTR', 'GTR', None, 'GTR', 'GTR', 'GTR',
       'GTR', 'GTR', 'GTR', 'GTR', None, 'GTR', 'GTR', 'GTR', 'GTR',
       'GTR', 'GTR', 'GTR', 'GTR', 'GTR', None, 'GTR', None, 'GTR', 'GTR',
       'GTR', 'GTR', 'GTR', None, 'GTR', 'GTR', 'GTR', 'STR', 'GTR',
       'STR', 'STR', 'GTR', 'GTR', 'GTR', None, 'GTR', None, 'GTR', 'GTR',
       'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', None,
       'GTR', 'GTR', 'GTR', None, 'GTR', 'GTR', 'GTR', 'GTR', 'GTR',
       'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR',
       'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR', 'GTR',
       'G

In [4]:
df['Extent_of_Resection'].fillna('Unknown', inplace=True)
df["Extent_of_Resection"].value_counts()


,count
Extent_of_Resection,
GTR,119
Unknown,107
STR,10


In [9]:
# Accelerate pandas operations with cuDF
%load_ext cudf.pandas

import os
import cupy as cp
import cudf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Enable GPU usage in TensorFlow and limit memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

def load_img(img_dir, img_list):
    images = []
    for i, image_name in enumerate(img_list):
        if image_name.split('.')[1] == 'npy':
            image = cp.asarray(np.load(img_dir + image_name))  # Load with cupy
            images.append(image)
    images = cp.array(images)  # Convert to cupy array
    return images

def load_img_batch(img_dir, img_list_batch):
    images = [cp.asarray(np.load(img_dir + image_name)) for image_name in img_list_batch if image_name.split('.')[1] == 'npy']
    return cp.array(images)

# Get survival data for training and testing sets (using pandas DataFrame)
survival_dict = dict(zip(df['image_path'].apply(lambda x: x.split('/')[-1]), df['Survival_days']))

def imageLoader(img_dir, img_list, mask_dir, mask_list, survival_dict):
    """
    Loads and preprocesses image and mask data, including survival data.

    Args:
        img_dir (str): Directory containing image files.
        img_list (list): List of image filenames.
        mask_dir (str): Directory containing mask files.
        mask_list (list): List of mask filenames.
        survival_dict (dict): Dictionary mapping image filenames to survival days.

    Returns:
        tf.data.Dataset: A dataset containing image, mask, and survival data.
    """
    # Create datasets from file paths
    image_paths = [img_dir + image_name for image_name in img_list if image_name.split('.')[1] == 'npy']
    mask_paths = [mask_dir + mask_name for mask_name in mask_list if mask_name.split('.')[1] == 'npy']

    image_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    mask_ds = tf.data.Dataset.from_tensor_slices(mask_paths)

    # Function to load and preprocess data
    def load_and_preprocess(image_path, mask_path):
        image = tf.py_function(lambda path: np.load(path.numpy()), [image_path], tf.float32)  # Load with NumPy
        mask = tf.py_function(lambda path: np.load(path.numpy()), [mask_path], tf.float32)  # Load with NumPy
        image.set_shape([128, 128, 128, 3])  # Set shape explicitly
        mask.set_shape([128, 128, 128, 4])  # Set shape explicitly

        # Get survival data - modified section
        survival_key = tf.py_function(lambda path: os.path.basename(path.numpy().decode()), [image_path], tf.string) # Get filename as tf.string
        survival = tf.constant(survival_dict[survival_key.numpy().decode()], dtype=tf.float32) # Get survival using filename

        return image, mask, survival

    # Apply the loading and preprocessing function
    dataset = tf.data.Dataset.zip((image_ds, mask_ds)).map(load_and_preprocess)

    return dataset

# Get image and mask file lists
img_dir = '/kaggle/working/input_data_3channels/images/'
mask_dir = '/kaggle/working/input_data_3channels/masks/'
img_list = os.listdir(img_dir)
mask_list = os.listdir(mask_dir)

# Split data into training and testing sets
train_img_list, test_img_list, train_mask_list, test_mask_list = train_test_split(
    img_list, mask_list, test_size=0.2, random_state=42
)

# Create data generators
# Create data generators
batch_size = 32  # Adjust batch size as needed
train_generator = imageLoader(img_dir, train_img_list, mask_dir, train_mask_list, survival_dict)
test_generator = imageLoader(img_dir, test_img_list, mask_dir, test_mask_list, survival_dict)
# Define output signature for tf.data.Dataset
output_signature = (
    tf.TensorSpec(shape=(None, 128, 128, 128, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(None, 128, 128, 128, 4), dtype=tf.float32),  # Mask data
    tf.TensorSpec(shape=(None,), dtype=tf.float32)  # Survival data
)

# Create tf.data.Dataset from generator, ensuring NumPy arrays are yielded
train_dataset = tf.data.Dataset.from_generator(
    lambda: ((cp.asnumpy(X), cp.asnumpy(Y), cp.asnumpy(S)) for X, Y, S in train_generator),
    output_signature=output_signature
).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: ((cp.asnumpy(X), cp.asnumpy(Y), cp.asnumpy(S)) for X, Y, S in test_generator),
    output_signature=output_signature
).prefetch(tf.data.AUTOTUNE)

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=(128, 128, 128, 3)),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu'),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Custom training loop
def train_step(images, masks, survival_data):
    with tf.GradientTape() as tape:
        predictions = model(images)  # Get model predictions
        loss = tf.keras.losses.mean_squared_error(survival_data, predictions)  # Calculate loss

    gradients = tape.gradient(loss, model.trainable_variables)  # Calculate gradients
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # Apply gradients
    return loss, predictions

# Training loop
epochs = 10
for epoch in range(epochs):
    for step, (images, masks, survival_data) in enumerate(train_dataset):
        loss, predictions = train_step(images, masks, survival_data)
        print(f"Epoch {epoch}, Step {step}, Loss: {loss.numpy()}")

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas
1 Physical GPUs, 1 Logical GPUs


AttributeError: in user code:

    File "<ipython-input-9-3a4022bd1024>", line 71, in load_and_preprocess  *
        survival = tf.constant(survival_dict[survival_key.numpy().decode()], dtype=tf.float32) # Get survival using filename

    AttributeError: 'SymbolicTensor' object has no attribute 'numpy'


In [5]:
import pandas as pd
fs=pd.read_csv("/survival_info.csv")
ft=pd.read_csv("/features.csv")

In [6]:
fs.columns

Index(['Brats20ID', 'Age', 'Survival_days', 'Extent_of_Resection'], dtype='object')

In [7]:
fs.head()

,Brats20ID,Age,Survival_days,Extent_of_Resection
0,BraTS20_Training_001,60.463,289,GTR
1,BraTS20_Training_002,52.263,616,GTR
2,BraTS20_Training_003,54.301,464,GTR
3,BraTS20_Training_004,39.068,788,GTR
4,BraTS20_Training_005,68.493,465,GTR


In [17]:
ft.head()

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_ZoneEntropy,wavelet-LLL_glszm_ZonePercentage,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength
0,v3.0.1,2.0.2,2.5.0,1.8.0,3.11.12,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {}, 'Wavelet': {}}",22d6b6b708291431d11a2f9aec754ae95929e436,3D,"(1.0, 1.0, 1.0)",...,1446.267227,0.000441,7.242620,0.392442,43.345687,0.765144,0.000280,6668.675995,0.075649,0.699780
1,v3.0.1,2.0.2,2.5.0,1.8.0,3.11.12,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {}, 'Wavelet': {}}",f4e834c16cba4ca842d6faadc6038b106c467ab8,3D,"(1.0, 1.0, 1.0)",...,325.226862,0.002353,7.169546,0.222781,6880.235021,2.873798,0.000202,1646.895341,0.094403,0.177072
2,v3.0.1,2.0.2,2.5.0,1.8.0,3.11.12,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {}, 'Wavelet': {}}",d8e5238cbd092aff21bbdabab1adc6c44a38f9ba,3D,"(1.0, 1.0, 1.0)",...,38.315631,0.014401,6.457374,0.039069,62907.148036,10.220222,0.000447,121.170030,0.024946,0.075707
3,v3.0.1,2.0.2,2.5.0,1.8.0,3.11.12,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {}, 'Wavelet': {}}",380231f3ac2c2364ac28922fc55c1547191ba5b2,3D,"(1.0, 1.0, 1.0)",...,365.387940,0.001906,7.237462,0.202314,954.892910,0.709354,0.001010,1125.153449,0.088047,0.716459
4,v3.0.1,2.0.2,2.5.0,1.8.0,3.11.12,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {}, 'Wavelet': {}}",24b92e9682e90d478e1154009ded29502f1c09a5,3D,"(1.0, 1.0, 1.0)",...,969.048362,0.000871,7.386366,0.321170,228.479231,0.445414,0.000710,4514.328550,0.089952,1.438716


In [9]:
ft.shape


(368, 873)

In [8]:
fs.shape

(236, 4)

In [10]:
fs['Brats20ID'] = fs['Brats20ID'].str.extract(r'(\d+)$').astype(int)
fs.head()

,Brats20ID,Age,Survival_days,Extent_of_Resection
0,1,60.463,289,GTR
1,2,52.263,616,GTR
2,3,54.301,464,GTR
3,4,39.068,788,GTR
4,5,68.493,465,GTR


In [24]:
ft.head()

,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,...,wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_ZoneEntropy,wavelet-LLL_glszm_ZonePercentage,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength
0,89.015128,0.0,1467.0,53794,57,0.557774,0.498310,53.815083,107.995227,75.006666,...,1446.267227,0.000441,7.242620,0.392442,43.345687,0.765144,0.000280,6668.675995,0.075649,0.699780
1,36.343722,0.0,917.0,112429,3,0.690800,0.592540,49.478573,83.502556,73.539105,...,325.226862,0.002353,7.169546,0.222781,6880.235021,2.873798,0.000202,1646.895341,0.094403,0.177072
2,14.482188,0.0,476.0,31713,14,0.603543,0.490471,40.812645,83.211069,84.433406,...,38.315631,0.014401,6.457374,0.039069,62907.148036,10.220222,0.000447,121.170030,0.024946,0.075707
3,63.385387,0.0,1163.0,21091,8,0.505092,0.431483,28.447987,65.930748,57.428216,...,365.387940,0.001906,7.237462,0.202314,954.892910,0.709354,0.001010,1125.153449,0.088047,0.716459
4,35.928207,0.0,1163.0,29514,17,0.679469,0.437081,31.292233,71.593670,60.033324,...,969.048362,0.000871,7.386366,0.321170,228.479231,0.445414,0.000710,4514.328550,0.089952,1.438716


In [25]:
fs.head()

,Brats20ID,Age,Survival_days,Extent_of_Resection
0,1,60.463,289,GTR
1,2,52.263,616,GTR
2,3,54.301,464,GTR
3,4,39.068,788,GTR
4,5,68.493,465,GTR


In [15]:
# prompt: remove col in ft that are not num values
import numpy as np
# Assuming 'ft' is your DataFrame
# Convert non-numeric columns to numeric, coercing errors to NaN
for col in ft.columns:
    if not pd.api.types.is_numeric_dtype(ft[col]):
        try:
            ft[col] = pd.to_numeric(ft[col], errors='coerce')
        except:
            pass  # Skip columns that can't be converted

# Drop columns that are still not numeric after conversion
ft = ft.select_dtypes(include=np.number)


In [16]:
# prompt: drop col that have nan as the only value

import pandas as pd
import numpy as np

# Assuming 'ft' is your DataFrame
# Identify columns with only NaN values
cols_to_drop = [col for col in ft.columns if ft[col].isnull().all()]

# Drop those columns
ft = ft.drop(columns=cols_to_drop)


In [27]:
ft["Brats20ID"]=fs["Brats20ID"]
ft.head()

,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,...,wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_ZoneEntropy,wavelet-LLL_glszm_ZonePercentage,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength,Brats20ID
0,89.015128,0.0,1467.0,53794,57,0.557774,0.498310,53.815083,107.995227,75.006666,...,0.000441,7.242620,0.392442,43.345687,0.765144,0.000280,6668.675995,0.075649,0.699780,1.0
1,36.343722,0.0,917.0,112429,3,0.690800,0.592540,49.478573,83.502556,73.539105,...,0.002353,7.169546,0.222781,6880.235021,2.873798,0.000202,1646.895341,0.094403,0.177072,2.0
2,14.482188,0.0,476.0,31713,14,0.603543,0.490471,40.812645,83.211069,84.433406,...,0.014401,6.457374,0.039069,62907.148036,10.220222,0.000447,121.170030,0.024946,0.075707,3.0
3,63.385387,0.0,1163.0,21091,8,0.505092,0.431483,28.447987,65.930748,57.428216,...,0.001906,7.237462,0.202314,954.892910,0.709354,0.001010,1125.153449,0.088047,0.716459,4.0
4,35.928207,0.0,1163.0,29514,17,0.679469,0.437081,31.292233,71.593670,60.033324,...,0.000871,7.386366,0.321170,228.479231,0.445414,0.000710,4514.328550,0.089952,1.438716,5.0


In [28]:
merged_df=pd.merge(fs,ft,on="Brats20ID")
merged_df.head()

,Brats20ID,Age,Survival_days,Extent_of_Resection,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,original_shape_Elongation,...,wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_ZoneEntropy,wavelet-LLL_glszm_ZonePercentage,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength
0,1,60.463,289,GTR,89.015128,0.0,1467.0,53794,57,0.557774,...,1446.267227,0.000441,7.242620,0.392442,43.345687,0.765144,0.000280,6668.675995,0.075649,0.699780
1,2,52.263,616,GTR,36.343722,0.0,917.0,112429,3,0.690800,...,325.226862,0.002353,7.169546,0.222781,6880.235021,2.873798,0.000202,1646.895341,0.094403,0.177072
2,3,54.301,464,GTR,14.482188,0.0,476.0,31713,14,0.603543,...,38.315631,0.014401,6.457374,0.039069,62907.148036,10.220222,0.000447,121.170030,0.024946,0.075707
3,4,39.068,788,GTR,63.385387,0.0,1163.0,21091,8,0.505092,...,365.387940,0.001906,7.237462,0.202314,954.892910,0.709354,0.001010,1125.153449,0.088047,0.716459
4,5,68.493,465,GTR,35.928207,0.0,1163.0,29514,17,0.679469,...,969.048362,0.000871,7.386366,0.321170,228.479231,0.445414,0.000710,4514.328550,0.089952,1.438716


In [29]:
merged_df.isna().sum()

,0
Brats20ID,0
Age,0
Survival_days,0
Extent_of_Resection,107
diagnostics_Image-original_Mean,0
...,...
wavelet-LLL_ngtdm_Busyness,0
wavelet-LLL_ngtdm_Coarseness,0
wavelet-LLL_ngtdm_Complexity,0
wavelet-LLL_ngtdm_Contrast,0


In [30]:
merged_df.isnull().sum()

,0
Brats20ID,0
Age,0
Survival_days,0
Extent_of_Resection,107
diagnostics_Image-original_Mean,0
...,...
wavelet-LLL_ngtdm_Busyness,0
wavelet-LLL_ngtdm_Coarseness,0
wavelet-LLL_ngtdm_Complexity,0
wavelet-LLL_ngtdm_Contrast,0


In [31]:
merged_df.shape

(236, 860)

In [32]:
merged_df['Extent_of_Resection'].fillna('Unknown', inplace=True)

<ipython-input-32-c15c66807095>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Extent_of_Resection'].fillna('Unknown', inplace=True)


In [33]:
merged_df.isna().sum().sum()


np.int64(0)

In [34]:
merged_df=pd.merge(fs,ft,on="Brats20ID")

In [35]:
#!pip install scikit-learn
#!pip install mlxtend

import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression  # Or any other suitable model

# Assuming you have merged_df loaded

# 1. Preprocessing:
# Convert 'Extent_of_Resection' to numerical using OneHot Encoding
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(merged_df[['Extent_of_Resection']])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Extent_of_Resection']))
merged_df = pd.concat([merged_df, encoded_df], axis=1)
merged_df = merged_df.drop(columns=['Extent_of_Resection'])

# 2. Filter Method (using ANOVA F-value, Mutual Information, or Chi-squared):
# Select top k features based on F-value for numerical features
selector_f = SelectKBest(f_classif, k=20)  # Select top 20 features
X_numerical = merged_df.drop(columns=['Survival_days'])  # Exclude target variable
y = merged_df['Survival_days']
X_numerical_new = selector_f.fit_transform(X_numerical, y)
selected_features_f = X_numerical.columns[selector_f.get_support()]

# 3. Wrapper Method (using Sequential Feature Selector):
# Use a classifier (e.g., Logistic Regression) and SFS for feature selection
sfs = SequentialFeatureSelector(LogisticRegression(max_iter=1000), # Increase max_iter
                                 k_features=10,  # Select top 10 features
                                 forward=True,
                                 floating=False,
                                 scoring='accuracy',  # Or any other suitable metric
                                 cv=3)  # Reduced to 3-fold cross-validation

X_combined = X_numerical[selected_features_f]  # Use only numerical features from filter method
sfs = sfs.fit(X_combined, y)
selected_features_sfs = list(sfs.k_feature_names_)

# 4. Combine Selected Features:
# Combine features selected from both methods
all_selected_features = list(set(selected_features_f) | set(selected_features_sfs))

# Add the essential features you mentioned
all_selected_features.extend(['Brats20ID', 'Age', 'Survival_days'])

# Final DataFrame with selected features:
final_df = merged_df[all_selected_features]

print("Selected Features:", final_df.columns)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [  3 859] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in

Selected Features: Index(['Extent_of_Resection_STR',
       'wavelet-LHH_glszm_SmallAreaHighGrayLevelEmphasis',
       'wavelet-LHH_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-HHL_glszm_LargeAreaHighGrayLevelEmphasis',
       'wavelet-HHL_glcm_Autocorrelation',
       'wavelet-LHH_glrlm_LongRunHighGrayLevelEmphasis',
       'wavelet-HHL_glszm_HighGrayLevelZoneEmphasis',
       'wavelet-LHH_gldm_HighGrayLevelEmphasis',
       'wavelet-HHH_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-LHH_glszm_HighGrayLevelZoneEmphasis',
       'wavelet-HHL_gldm_HighGrayLevelEmphasis',
       'wavelet-LHH_glrlm_ShortRunHighGrayLevelEmphasis',
       'wavelet-HHH_glcm_ClusterShade',
       'wavelet-LHH_glrlm_HighGrayLevelRunEmphasis',
       'wavelet-HLH_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-HHL_glrlm_LongRunHighGrayLevelEmphasis',
       'wavelet-HHL_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-HHL_glrlm_HighGrayLevelRunEmphasis',
      

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
final_df.to_csv("final_df.csv",index=False)

In [38]:
final_df.columns

Index(['Extent_of_Resection_STR',
       'wavelet-LHH_glszm_SmallAreaHighGrayLevelEmphasis',
       'wavelet-LHH_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-HHL_glszm_LargeAreaHighGrayLevelEmphasis',
       'wavelet-HHL_glcm_Autocorrelation',
       'wavelet-LHH_glrlm_LongRunHighGrayLevelEmphasis',
       'wavelet-HHL_glszm_HighGrayLevelZoneEmphasis',
       'wavelet-LHH_gldm_HighGrayLevelEmphasis',
       'wavelet-HHH_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-LHH_glszm_HighGrayLevelZoneEmphasis',
       'wavelet-HHL_gldm_HighGrayLevelEmphasis',
       'wavelet-LHH_glrlm_ShortRunHighGrayLevelEmphasis',
       'wavelet-HHH_glcm_ClusterShade',
       'wavelet-LHH_glrlm_HighGrayLevelRunEmphasis',
       'wavelet-HLH_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-HHL_glrlm_LongRunHighGrayLevelEmphasis',
       'wavelet-HHL_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-HHL_glrlm_HighGrayLevelRunEmphasis',
       'wavelet-LLL_glszm

In [39]:
final_df.isna().sum()

,0
Extent_of_Resection_STR,0
wavelet-LHH_glszm_SmallAreaHighGrayLevelEmphasis,0
wavelet-LHH_gldm_LargeDependenceHighGrayLevelEmphasis,0
wavelet-HHL_glszm_LargeAreaHighGrayLevelEmphasis,0
wavelet-HHL_glcm_Autocorrelation,0
wavelet-LHH_glrlm_LongRunHighGrayLevelEmphasis,0
wavelet-HHL_glszm_HighGrayLevelZoneEmphasis,0
wavelet-LHH_gldm_HighGrayLevelEmphasis,0
wavelet-HHH_gldm_LargeDependenceHighGrayLevelEmphasis,0
wavelet-LHH_glszm_HighGrayLevelZoneEmphasis,0


In [43]:
final_df.dtypes

,0
Extent_of_Resection_STR,float64
wavelet-LHH_glszm_SmallAreaHighGrayLevelEmphasis,float64
wavelet-LHH_gldm_LargeDependenceHighGrayLevelEmphasis,float64
wavelet-HHL_glszm_LargeAreaHighGrayLevelEmphasis,float64
wavelet-HHL_glcm_Autocorrelation,float64
wavelet-LHH_glrlm_LongRunHighGrayLevelEmphasis,float64
wavelet-HHL_glszm_HighGrayLevelZoneEmphasis,float64
wavelet-LHH_gldm_HighGrayLevelEmphasis,float64
wavelet-HHH_gldm_LargeDependenceHighGrayLevelEmphasis,float64
wavelet-LHH_glszm_HighGrayLevelZoneEmphasis,float64


In [56]:
final_df.drop(columns=["Brats20ID"],inplace=True)

In [82]:
final_df.columns

Index(['Extent_of_Resection_STR',
       'wavelet-LHH_glszm_SmallAreaHighGrayLevelEmphasis',
       'wavelet-LHH_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-HHL_glszm_LargeAreaHighGrayLevelEmphasis',
       'wavelet-HHL_glcm_Autocorrelation',
       'wavelet-LHH_glrlm_LongRunHighGrayLevelEmphasis',
       'wavelet-HHL_glszm_HighGrayLevelZoneEmphasis',
       'wavelet-LHH_gldm_HighGrayLevelEmphasis',
       'wavelet-HHH_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-LHH_glszm_HighGrayLevelZoneEmphasis',
       'wavelet-HHL_gldm_HighGrayLevelEmphasis',
       'wavelet-LHH_glrlm_ShortRunHighGrayLevelEmphasis',
       'wavelet-HHH_glcm_ClusterShade',
       'wavelet-LHH_glrlm_HighGrayLevelRunEmphasis',
       'wavelet-HLH_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-HHL_glrlm_LongRunHighGrayLevelEmphasis',
       'wavelet-HHL_gldm_LargeDependenceHighGrayLevelEmphasis',
       'wavelet-HHL_glrlm_HighGrayLevelRunEmphasis',
       'wavelet-LLL_glszm

In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression  # Or other regression models
from sklearn.metrics import mean_squared_error, r2_score

# Assuming you have final_df loaded

# Split data into features (X) and target (y)
X = final_df.drop(columns=['Survival_days'])
y = final_df['Survival_days']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a regression model (e.g., Linear Regression)
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(y_test,y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")




69      375.0
208     519.0
181      82.0
9       515.0
128     692.0
110     334.0
197     232.0
94      353.0
224     311.0
15      368.0
226     184.0
55      208.0
229     456.0
30      170.0
202     244.0
148     804.0
19       55.0
113     613.0
227     197.0
24       23.0
210     147.0
10      421.0
96       21.0
145     199.0
222     351.0
25      203.0
87      436.0
184      22.0
85      510.0
18      287.0
147     600.0
66      387.0
45      318.0
140      86.0
126     374.0
171     828.0
165     329.0
16      439.0
157     394.0
178     946.0
162      71.0
205     508.0
125    1410.0
60     1592.0
200       5.0
231      62.0
82      401.0
Name: Survival_days, dtype: float64 [8.04516082e+02 6.80657829e+02 1.05406797e+02 7.51860328e+02
 6.65614200e+02 3.78865789e+02 1.55507999e+02 2.57387564e+02
 1.47391328e+02 2.37664229e+02 2.09675765e+02 1.99536265e+02
 7.35792853e+02 1.10455088e+04 1.59719880e+02 7.13601774e+02
 2.59141817e+02 6.98493647e+02 2.60756312e+02 2.11013953e+02
 

In [58]:
#save model
import pickle

with open('survivaldays.pkl', 'wb') as f:
    pickle.dump(model, f)
    print("Model saved successfully!")

Model saved successfully!


In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression  # Or other classification models
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have final_df loaded

# Create a binary target variable for survival chance (e.g., survival > threshold)
threshold = final_df['Survival_days'].median()  # Or any other relevant threshold
final_df['Survival_chance'] = (final_df['Survival_days'] > threshold).astype(int)

# Split data into features (X) and target (y)
X = final_df.drop(columns=['Survival_days', 'Survival_chance'])
y = final_df['Survival_chance']

# Split data into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a classification model (e.g., Logistic Regression)
model1 = LogisticRegression()
model1.fit(X_train, y_train)

# Make predictions on the test set
y_pred1 = model1.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test1, y_pred1)
report = classification_report(y_test1, y_pred1)
print(y_test1,y_pred1)
print(f"Accuracy: {accuracy}")
print(report)

69     1
208    1
181    0
9      1
128    1
110    0
197    0
94     0
224    0
15     0
226    0
55     0
229    1
30     0
202    0
148    1
19     0
113    1
227    0
24     0
210    0
10     1
96     0
145    0
222    0
25     0
87     1
184    0
85     1
18     0
147    1
66     1
45     0
140    0
126    1
171    1
165    0
16     1
157    1
178    1
162    0
205    1
125    1
60     1
200    0
231    0
82     1
Name: Survival_chance, dtype: int64 [0 1 1 0 1 1 1 0 0 1 1 0 1 1 0 0 1 0 0 0 0 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 0 1 0 1 0 1 1]
Accuracy: 0.5106382978723404
              precision    recall  f1-score   support

           0       0.59      0.38      0.47        26
           1       0.47      0.67      0.55        21

    accuracy                           0.51        47
   macro avg       0.53      0.53      0.51        47
weighted avg       0.53      0.51      0.50        47



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [60]:
import pickle

with open('survivalchance.pkl', 'wb') as f:
    pickle.dump(model, f)
    print("Model saved successfully!")

Model saved successfully!


In [71]:
# prompt: tf version
import tensorflow as tf
tf.__version__


'2.18.0'

In [95]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
# Add other potential models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Assuming you have final_df loaded and preprocessed as before

# 1. Feature Scaling:
scaler = StandardScaler()
X = final_df.drop(columns=['Survival_days','Survival_chance'])
X_scaled = scaler.fit_transform(X)  # Scale features

# 2. Split data into training and testing sets:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, final_df['Survival_days'], test_size=0.2, random_state=42)

# 3. Model Selection and Hyperparameter Tuning:
# Try different models and adjust hyperparameters
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),  # Adjust alpha
    "Lasso Regression": Lasso(alpha=1.0),  # Adjust alpha
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42), # Adjust n_estimators
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42) # Adjust n_estimators
}

best_model = None
best_r2 = -float('inf')  # Initialize with negative infinity

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)

    print(f"{model_name}: R-squared = {r2}")

    if r2 > best_r2:
        best_r2 = r2
        best_model = model
        best_model_name = model_name

print(f"\nBest Model: {best_model_name} with R-squared = {best_r2}")

# Evaluate the best model using MSE as well
y_pred_best = best_model.predict(X_test)
mse_best = mean_squared_error(y_test, y_pred_best)
print(f"Best Model MSE: {mse_best}")
import joblib
joblib.dump(best_model, "best_modelsurvival.pkl") # changed to use joblib and .pkl extension

Linear Regression: R-squared = -8569.90857621482
Ridge Regression: R-squared = -0.03636142041184742
Lasso Regression: R-squared = -2.7108376209101266
Random Forest: R-squared = -0.20305763962774548
Gradient Boosting: R-squared = -0.2641819957796703

Best Model: Ridge Regression with R-squared = -0.03636142041184742
Best Model MSE: 106854.53827083726


['best_modelsurvival.pkl']

In [81]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler, RobustScaler
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE  # For SMOTE
import warnings
warnings.filterwarnings("ignore")


# Data Augmentation with SMOTE:
def augment_data_smote(X, y):
    """Augments the data using SMOTE."""
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    return X_resampled, y_resampled

# Data Augmentation with Mixup:
def augment_data_mixup(X, y, alpha=0.2):
    """Augments the data using Mixup."""
    num_samples = X.shape[0]
    X_mixup = []
    y_mixup = []

    for i in range(num_samples):
        j = np.random.randint(0, num_samples)  # Randomly select another sample
        lam = np.random.beta(alpha, alpha)  # Sample lambda from Beta distribution
        X_mixup.append(lam * X[i] + (1 - lam) * X[j])
        y_mixup.append(lam * y[i] + (1 - lam) * y[j])

    X_mixup = np.array(X_mixup)
    y_mixup = np.array(y_mixup)
    return np.concatenate([X, X_mixup]), np.concatenate([y, y_mixup])


# Assuming you have 'final_df' loaded and preprocessed



# 2. Split data:
X = final_df.drop(columns=['Survival_days'])
y = final_df['Survival_days']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Data Augmentation:
# Choose one or combine data augmentation techniques:
# X_train_augmented, y_train_augmented = augment_data_smote(X_train, y_train)  # SMOTE
X_train_augmented, y_train_augmented = augment_data_mixup(X_train.values, y_train.values)  # Mixup

# 4. Model Selection and Hyperparameter Tuning:
# Define a pipeline with scaling and model:
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Choose appropriate scaler: StandardScaler, MinMaxScaler, RobustScaler
    ('poly', PolynomialFeatures(degree=2)),  # Optional: Add polynomial features
    ('model', MLPRegressor(random_state=42))  # Placeholder for the model
])

# Define models and their hyperparameter grids:
models = {
    "Linear Regression": (LinearRegression(), {}),
    "Ridge Regression": (Ridge(random_state=42), {'model__alpha': [0.1, 1.0, 10.0]}),
    "Lasso Regression": (Lasso(random_state=42), {'model__alpha': [0.1, 1.0, 10.0]}),
    "Random Forest": (RandomForestRegressor(random_state=42), {'model__n_estimators': [50, 100, 200]}),
    "Gradient Boosting": (GradientBoostingRegressor(random_state=42), {'model__n_estimators': [50, 100, 200]}),
    "Neural Network": (MLPRegressor(random_state=42), {
        'model__hidden_layer_sizes': [(100, 50), (64, 32), (128, 64, 32)],
        'model__activation': ['relu', 'tanh'],
        'model__solver': ['adam'],
        'model__alpha': [0.0001, 0.001, 0.01],
        'model__learning_rate_init': [0.001, 0.01],
        'model__max_iter': [500, 1000],
        'model__early_stopping': [True]
    })
}

# Define cross-validation strategy:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

best_model = None
best_r2 = -float('inf')

# Iterate through models and tune hyperparameters:
for model_name, (model, param_grid) in models.items():
    pipeline.set_params(model=model)  # Set the model in the pipeline
    grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='r2', verbose=2, n_jobs=-1)
    grid_search.fit(X_train_augmented, y_train_augmented)

    # Evaluate the best model from the grid search:
    y_pred = grid_search.best_estimator_.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print(f"{model_name}:")
    print(f"  R-squared = {r2}")
    print(f"  MSE = {mse}")
    print("Best parameters:", grid_search.best_params_)

    if r2 > best_r2:
        best_r2 = r2
        best_model = grid_search.best_estimator_
        best_model_name = model_name

print(f"\nBest Model: {best_model_name} with R-squared = {best_r2}")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Linear Regression:
  R-squared = -161926182436631.47
  MSE = 1.6695476228119079e+19
Best parameters: {}
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Ridge Regression:
  R-squared = -178080.45051316047
  MSE = 18361172844.144196
Best parameters: {'model__alpha': 10.0}
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Lasso Regression:
  R-squared = -279896.6420418756
  MSE = 28858979806.094296
Best parameters: {'model__alpha': 10.0}
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Random Forest:
  R-squared = 0.2705246451649095
  MSE = 75212.90419116773
Best parameters: {'model__n_estimators': 200}
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Gradient Boosting:
  R-squared = 0.23497672082792254
  MSE = 78878.08987514071
Best parameters: {'model__n_estimators': 200}
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Neural Network:
  R-squared = -5878.10588717563
  MSE

In [97]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer  # For handling missing values
from sklearn.neural_network import MLPRegressor  # For Neural Networks
# For Support Vector Regression (optional)
from sklearn.svm import SVR

# Assuming you have 'final_df' loaded

# 1. Handle Missing Values:
imputer = SimpleImputer(strategy='mean')  # Or 'median', 'most_frequent'
final_df_imputed = pd.DataFrame(imputer.fit_transform(final_df), columns=final_df.columns)

# 2. Feature Engineering (Optional but recommended):
# - Create new features or interactions if you think they might be relevant
# - Example: Create interaction terms between Age and tumor size
# final_df_imputed['Age_TumorSize_Interaction'] = final_df_imputed['Age'] * final_df_imputed['Tumor_Size'] # Replace with actual feature names

# 3. Feature Scaling:
scaler = StandardScaler()  # or MinMaxScaler, RobustScaler
X = final_df_imputed.drop(columns=['Survival_days','Survival_chance'])
X_scaled = scaler.fit_transform(X)

# 4. Split data:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, final_df_imputed['Survival_days'], test_size=0.2, random_state=42)

# 5. Data Augmentation (Optional - adjust as needed):
# Check unique values and counts of the target variable
unique_values, counts = np.unique(y_train, return_counts=True)

# Print unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"Value: {value}, Count: {count}")

try:
    X_train_augmented, y_train_augmented = SMOTE(random_state=42, k_neighbors=min(5, min(counts) - 1)).fit_resample(X_train, y_train)
except ValueError as e:
    print("Error during SMOTE:", e)
    print("Skipping SMOTE due to insufficient samples in one or more classes.")
    X_train_augmented, y_train_augmented = X_train, y_train  # Use original data if SMOTE fails

X_train = X_train_augmented
y_train = y_train_augmented

# 6. Model Selection and Hyperparameter Tuning:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, interaction_only=True)),
    ('model', LinearRegression())
])

models = {
    "Linear Regression": (LinearRegression(), {}),
    "Ridge Regression": (Ridge(random_state=42), {'model__alpha': [0.1, 1.0, 10.0]}),
    "Lasso Regression": (Lasso(random_state=42), {'model__alpha': [0.1, 1.0, 10.0]}),
    "Random Forest": (RandomForestRegressor(random_state=42), {'model__n_estimators': [50, 100, 200]}),
    "Gradient Boosting": (GradientBoostingRegressor(random_state=42), {'model__n_estimators': [50, 100, 200]}),
    # Add more complex models:
    "Neural Network": (MLPRegressor(random_state=42, early_stopping=True), {
        'model__hidden_layer_sizes': [(100, 50), (64, 32), (128, 64, 32)],
        'model__activation': ['relu', 'tanh'],
        'model__solver': ['adam'],
        'model__alpha': [0.0001, 0.001, 0.01],
        'model__learning_rate_init': [0.001, 0.01],
        'model__max_iter': [200, 500]
    }),
    # Optional: Support Vector Regression
    "Support Vector Regression": (SVR(), {
       'model__kernel': ['linear', 'rbf'],
         'model__C': [0.1, 1, 10],
        'model__epsilon': [0.01, 0.1, 1]
     })
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

best_model = None
best_r2 = -float('inf')

for model_name, (model, param_grid) in models.items():
    pipeline.set_params(model=model)
    grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='r2', verbose=2, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    y_pred = grid_search.best_estimator_.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print(f"{model_name}:")
    print(f"  R-squared = {r2}")
    print(f"  MSE = {mse}")
    print("Best parameters:", grid_search.best_params_)

    if r2 > best_r2:
        best_r2 = r2
        best_model = grid_search.best_estimator_
        best_model_name = model_name

print(f"\nBest Model: {best_model_name} with R-squared = {best_r2}")

# 7. Predict Survival Days and Probability:
y_pred_best = best_model.predict(X_test)
max_survival_time = 1000  # Adjust if needed
survival_probability = (y_pred_best / max_survival_time) * 100
print(f"Survival Probabilities: {survival_probability}")

Value: 12.0, Count: 1
Value: 30.0, Count: 2
Value: 32.0, Count: 1
Value: 33.0, Count: 1
Value: 50.0, Count: 1
Value: 56.0, Count: 1
Value: 58.0, Count: 1
Value: 67.0, Count: 1
Value: 74.0, Count: 1
Value: 77.0, Count: 2
Value: 78.0, Count: 1
Value: 80.0, Count: 1
Value: 82.0, Count: 2
Value: 84.0, Count: 1
Value: 85.0, Count: 1
Value: 89.0, Count: 1
Value: 99.0, Count: 1
Value: 103.0, Count: 1
Value: 104.0, Count: 1
Value: 105.0, Count: 1
Value: 106.0, Count: 1
Value: 110.0, Count: 1
Value: 111.0, Count: 1
Value: 112.0, Count: 1
Value: 114.0, Count: 1
Value: 117.0, Count: 1
Value: 120.0, Count: 1
Value: 121.0, Count: 1
Value: 127.0, Count: 1
Value: 139.0, Count: 1
Value: 142.0, Count: 1
Value: 145.0, Count: 1
Value: 146.0, Count: 1
Value: 150.0, Count: 1
Value: 152.0, Count: 1
Value: 153.0, Count: 1
Value: 155.0, Count: 1
Value: 168.0, Count: 1
Value: 169.0, Count: 1
Value: 172.0, Count: 1
Value: 175.0, Count: 1
Value: 180.0, Count: 1
Value: 182.0, Count: 1
Value: 187.0, Count: 1
Value

In [98]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

# Define a wider range of hyperparameters
param_grid = {
    'model__C': [0.1, 1, 10, 100],
    'model__epsilon': [0.01, 0.1, 1, 10],
    'model__kernel': ['linear', 'rbf', 'poly'],
    'model__gamma': ['scale', 'auto']  # If using 'rbf' or 'poly'
}

# Create and train the GridSearchCV object (using your existing pipeline)
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and model performance
print("Best parameters:", grid_search.best_params_)
print("Best R-squared:", grid_search.best_score_)

# Get the best model and make predictions
best_svr_model = grid_search.best_estimator_
y_pred = best_svr_model.predict(X_test)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best parameters: {'model__C': 100, 'model__epsilon': 10, 'model__gamma': 'auto', 'model__kernel': 'rbf'}
Best R-squared: -0.01546544386389923


In [87]:
#save model
import joblib
joblib.dump(best_model, "best_modelsurvivalchance.pkl") # changed to use joblib and .pkl extension

['best_modelsurvivalchance.pkl']